In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding, Input
from tensorflow.keras.preprocessing.sequence import pad_sequences
from transformers import BertTokenizer, TFBertModel
import tensorflow as tf

In [3]:
# Load pre-trained BERT model and tokenizer from Hugging Face
model_name = "GerMedBERT/medbert-512"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = TFBertModel.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/354 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/246k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/717k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/681 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['bert.embeddings.position_ids', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFBertModel were not initialized from the PyTorch model and are newly initialized: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
You should probably TRAIN this model on a down-

In [4]:
# Function to generate word embeddings for a given text
def generate_bert_embeddings(text, tokenizer, model):
    inputs = tokenizer(text, return_tensors="tf", truncation=True, padding=True)
    outputs = model(inputs)
    embeddings = outputs.last_hidden_state
    return embeddings

In [5]:
# Example text
text = "April is the fourth month of the year."
bert_embeddings = generate_bert_embeddings(text, tokenizer, model)
print(bert_embeddings.shape)

(1, 16, 768)


In [6]:
texts = ["April is the fourth month of the year.", "May is the fifth month of the year."]
labels = [0, 1]

bert_embeddings = [generate_bert_embeddings(text, tokenizer, model).numpy().squeeze(0) for text in texts]
padded_embeddings = pad_sequences(bert_embeddings, padding='post', dtype='float32')

# LSTM model
model = Sequential([
    Input(shape=(padded_embeddings.shape[1], 768)),
    LSTM(128),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

# Train the model
model.fit(padded_embeddings, labels, epochs=10)

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 128)            │       459,264 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 459,393 (1.75 MB)

 Trainable params: 459,393 (1.75 MB)

 Non-trainable params: 0 (0.00 B)

ValueError: Unrecognized data type: x=[[[ 0.04710687  0.22355315  0.32865202 ...  0.14702877 -1.0112246
   -0.72213984]
  [-1.5348815   0.29237837  0.5277895  ... -0.9500351   0.16193624
   -0.2630154 ]
  [-0.804341    1.5015209  -0.28092915 ...  0.35476714  2.1031134
   -1.5111918 ]
  ...
  [-0.07200021  0.32776937  0.75565195 ... -0.51212895 -1.091701
   -1.2017175 ]
  [-1.4916472  -0.6323696   0.18549556 ...  0.37174794  0.2951508
   -0.2637041 ]
  [-0.94521135  0.17419462  0.46114913 ... -0.28270206  1.3198467
   -1.3205004 ]]

 [[ 0.0735783   0.06232324  0.24887645 ...  0.15773608 -1.0024579
   -0.5637912 ]
  [-2.2064016   0.28758135  0.88242507 ... -1.5954025   0.89686817
    0.1457639 ]
  [-0.45352536  1.1928525  -0.21850574 ...  0.44245172  2.1628232
   -0.9994272 ]
  ...
  [-0.3267316   0.05329883  0.5749561  ... -0.32687896 -1.0044115
   -0.95092803]
  [-1.2204345  -1.0836145   0.17642064 ... -0.05053614  0.17941532
    0.06404392]
  [-1.3598998  -0.29793477  1.0684358  ... -